# Prepare Sequential Files and Patterns

Uses:

- resources/historical_join.xlsx (HISTORICAL_FILE_JOIN from prepare/1.historical.ipynb)

Generates:

- resources/seq_patterns/sequencial_project.txt
- resources/seq_patterns/input_sequencial_init_in_out.txt
- resources/seq_patterns/input_sequencial.txt


**See manual instructions in the end of the notebook to generate other files:**

- resources/seq_patterns/output_tam1.txt
- resources/seq_patterns/output_tam3.txt
- resources/seq_patterns/output_tam4.txt
- resources/seq_patterns/countpatterns_OutX_notOutx_Xlastslice.csv

In [1]:
import sys  
sys.path.insert(1, '..')

In [2]:
import pandas as pd
import numpy as np
import os
from itertools import groupby

import matplotlib.pyplot as plt 

from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR, SEQ_PATTERNS_DIR

In [3]:
FIRST_DB_COLUMN = 7

This script converts the historical_join dataset to the file format required by the SPMF library. At the end, it gives instructions for generating sequential patterns using the SPMF.jar file located in db-mining/resources/seq_patterns.

In [4]:
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)
len(df)

31116

In [5]:
df

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0


Rename columns to be more consistent. Transform both spaces and '-' into '_'

In [6]:
#df2.rename(columns={'MS Access': 'MS_Access', 'IBM DB2': 'IBM_DB2','Cosmos BD':'Cosmos_BD','MS SQL Server':'MS_SQL_Server'}, inplace=True)
df.rename(columns={'Join_Ignite-Sql':'Join_Ignite_Sql','Join_Ignite-NoSql':'Join_Ignite_NoSql','Join_MarkLogic-Sql':'Join_MarkLogic_Sql',
                    'Join_MarkLogic-NoSql':'Join_MarkLogic_NoSql','Join_Virtuoso-Sql':'Join_Virtuoso_Sql','Join_Virtuoso-NoSql':'Join_Virtuoso_NoSql'}, inplace=True)
df.rename(columns={'Microsoft Azure Table Storage': 'MicrosoftAzureTableStorage'}, inplace=True)
df.rename(columns={'IBM DB2':'IBM_DB2','Firebase Realtime':'FirebaseRealtime'}, inplace=True)
df.rename(columns={'Influx DB':'Influx_DB', 'Amazon Redshift':'AmazonRedshift','MS SQL Server': 'MS_SQL_Server'}, inplace=True)
df.rename(columns={'Microsoft Azure CosmosDB':'MicrosoftAzureCosmosDB', 'MS Access':'MS_Access','SAP Adaptive Server':'SAPAdaptiveServer'}, inplace=True)
df.rename(columns={'Google Cloud Datastore': 'GoogleCloudDatastore','Riak KV':'Riak_KV', 'Google BigQuery':'GoogleBigQuery'}, inplace=True)
df.rename(columns={'MySQL_Maria DB': 'MySQL_MariaDB', 'MS SQL Server_Microsoft Azure SQL Database': 'MSSQLServer_MicrosoftAzureSQLDatabase', 'Kdb+': 'Kdb'}, inplace=True)
df.rename(columns={'SAP SQL Anywhere': 'SAPSQLAnywhere', 'PostgreSQL_ CockroachDB': 'PostgreSQL_CockroachDB'}, inplace=True)

print(df.columns)

Index(['owner', 'name', 'domain', 'sha1', 'part_commit', 'date_commit',
       'isLast', 'SapHana', 'Realm', 'SAPSQLAnywhere', 'FileMaker', 'ArangoDB',
       'Informix', 'HyperSQL', 'CouchDB', 'Riak_KV', 'Neo4j', 'Singlestore',
       'Snowflake', 'MarkLogic', 'Hazelcast', 'GoogleCloudFirestore',
       'Influx_DB', 'H2', 'Interbase', 'Teradata', 'Oracle', 'Impala',
       'MicrosoftAzureCosmosDB', 'MySQL', 'MS_Access',
       'MicrosoftAzureTableStorage', 'Aerospike', 'IBM_DB2', 'DynamoDB',
       'Vertica', 'Ingres', 'MongoDB', 'MS_SQL_Server', 'GoogleCloudDatastore',
       'FirebaseRealtime', 'SQLite', 'PostgreSQL', 'SAPAdaptiveServer',
       'ClickHouse', 'Kdb', 'HBase', 'Firebird', 'Cassandra', 'Couchbase',
       'Etcd', 'PostGIS', 'Redis', 'Join_Ignite_Sql', 'Join_Ignite_NoSql',
       'Join_Virtuoso_Sql', 'Join_Virtuoso_NoSql'],
      dtype='object')


In [7]:
[x for x in df.columns if ' ' in x or '-' in x]

[]

In [8]:
df2 = df.copy()
print(df2.columns)

Index(['owner', 'name', 'domain', 'sha1', 'part_commit', 'date_commit',
       'isLast', 'SapHana', 'Realm', 'SAPSQLAnywhere', 'FileMaker', 'ArangoDB',
       'Informix', 'HyperSQL', 'CouchDB', 'Riak_KV', 'Neo4j', 'Singlestore',
       'Snowflake', 'MarkLogic', 'Hazelcast', 'GoogleCloudFirestore',
       'Influx_DB', 'H2', 'Interbase', 'Teradata', 'Oracle', 'Impala',
       'MicrosoftAzureCosmosDB', 'MySQL', 'MS_Access',
       'MicrosoftAzureTableStorage', 'Aerospike', 'IBM_DB2', 'DynamoDB',
       'Vertica', 'Ingres', 'MongoDB', 'MS_SQL_Server', 'GoogleCloudDatastore',
       'FirebaseRealtime', 'SQLite', 'PostgreSQL', 'SAPAdaptiveServer',
       'ClickHouse', 'Kdb', 'HBase', 'Firebird', 'Cassandra', 'Couchbase',
       'Etcd', 'PostGIS', 'Redis', 'Join_Ignite_Sql', 'Join_Ignite_NoSql',
       'Join_Virtuoso_Sql', 'Join_Virtuoso_NoSql'],
      dtype='object')


Generating the dictionary with the names of the databases

In [9]:
sorted_df = df2.sort_values(by=["owner", "name", "part_commit"])

m_init = lambda index: index*10000
m_in = lambda index: index*1000
m_current = lambda index: index
m_out = lambda index: index*100

i = 0
numeric_lines = []
named_lines = []
linhaproj = []
for (owner, name), rows in groupby(sorted_df.iterrows(), key=lambda row: (row[1]["owner"], row[1]["name"])):
    numeric_line = []
    named_line = []
    last_numeric_slice = []
    any_db = False
    for row_index, (_, row) in enumerate(rows):
        current_numeric_slice = []
        current_named_slice = []
        for index, col in enumerate(df2.columns):
            if index < FIRST_DB_COLUMN: continue
            if row[col] == 1:  # Found in current slice
                current_named_slice.append(col)
                current = m_current(index)
                if row_index == 0:  # Is the first slice
                    current_numeric_slice.append(m_init(index))
                elif current not in last_numeric_slice:  # Was inserted in the current slice
                    current_numeric_slice.append(m_in(index))
                current_numeric_slice.append(current)
            elif m_current(index) in last_numeric_slice: # Was removed in the current slices
                current_numeric_slice.append(m_out(index))
        numeric_line += current_numeric_slice + ['-1'] #(['-1'] if current_slice else [])
        named_line += current_named_slice + ['-1']
        any_db |= bool(current_numeric_slice)
        last_numeric_slice = current_numeric_slice
    if any_db:
        numeric_line[-1] += " -2"
        named_line[-1] += " -2"
        
        numeric_lines.append(numeric_line)
        named_lines.append(named_line)
        linhaproj.append((owner, name))
        i+=1

## Generate sequential files

In [10]:
with open(SEQ_PATTERNS_DIR + os.sep + 'sequencial_project.txt', "w") as output:
    output.write(str(linhaproj))             
                
dict_col=['@CONVERTED_FROM_TEXT']
for index, col in enumerate(df2.columns):
    #linha=[]
    if index >= FIRST_DB_COLUMN: 
        dict_col.append(f'@ITEM={index*10000}=INIT{col}')
        dict_col.append(f'@ITEM={index*1000}=IN{col}')
        dict_col.append(f'@ITEM={index}={col}')
        dict_col.append(f'@ITEM={index*100}=OUT{col}')
        
dict_col.append('@ITEM=-1=|')

with open(SEQ_PATTERNS_DIR + os.sep + 'input_sequencial_init_in_out.txt', "w") as file:
    file.write('\n'.join(dict_col) + '\n')
    for line in numeric_lines:
        file.write(' '.join(map(str, line)) + '\n')
        
with open(SEQ_PATTERNS_DIR + os.sep + 'input_sequencial.txt', "w") as file:
    for line in named_lines:
        file.write(' '.join(map(str, line)) + '\n')
        

## Generate Sequential Patterns using SPMF

Now we have the input file in the format required by the SPMF library. To generate sequential patterns according to our analysis, via the command line, as follows the steps below:: in a folder containing spmf.jar and the output file output__in_out.txt, in this case in /resources/seq_patterns. 

a) To generate results in size 1:

 Run "java -jar spmf.jar run PrefixSpan input_sequencial_init_in_out.txt output_tam1.txt 1% 1" 

b) To generate results in size 3:

Run "java -jar spmf.jar run PrefixSpan input_sequencial_init_in_out.txt output_tam3.txt 1% 3" 

c) To generate results in size 4:

Run "java -jar spmf.jar run PrefixSpan input_sequencial_init_in_out.txt output_tam4_sid.txt 1% 4" 

d) Save the three output files in the resources/seq_pattens folder as they will be used by "historical_seqpatterns_filters.ipynb" script.

 
PrefixSpan is one of the fastest sequential pattern mining algorithm (http://www.philippe-fournier-viger.com/spmf/PrefixSpan.php). 
The PrefixSpan implementation allows to specify additional optional parameter(s) : 

    "maximum pattern length" allows to specify the maximum number of items that patterns found should contain. 

    "show sequences ids?" (true/false) This parameter allows to specify that sequence ids of sequences containing a pattern should be output for each pattern found.
     For example, if the parameter is set to true, each pattern in the output file will be followed by the keyword #SID followed by a list of sequences ids (integers separated by space). 
     For example, a line terminated by "#SID: 0 2" means that the pattern on this line appears in the first and the third sequences of the sequence database (sequences with ids 0 and 2). 
     Run "java -jar spmf.jar run PrefixSpan input_sequencial_init_in_out.txt output_tam4_sid.txt 1% 4 true"

## Count Specific Sequential Patterns using patterncounter

How to generate `countpatterns_OutX_notOutx_Xlastslice.csv`:

Use the Pattern counter tool with the following command: `$ patterncounter convert -f input_sequencial_init_in_out.txt | patterncounter count --csv -n "InX" -v X + "InX -> OutX" -v X + "InX -> OutX -> InX" -v X + "InX -> OutX -> InX -> OutX" -v X + "InX -> OutX -> InX -> OutX -> InX" -v X + "InX -> OutX -> InX -> OutX -> InX -> OutX" -v X + "InX -> OutX -> InX -> OutX -> InX -> OutX -> InX" -v X + "InX -> OutX -> InX -> OutX -> InX -> OutX -> InX -> OutX" -v X + "OutX" -v X + "X ~OutX" -v X + '$X' -v X > countpatterns_OutX_notOutx_Xlastslice.csv`

- InX : Selects the BDs that entered at least once
- InX -> OutX : Selects the BDs that left at least once
- InX -> OutX -> InX : Selects the BDs that entered at least twice
- InX -> OutX -> InX -> OutX : Selects the BDs that left at least twice
- InX -> OutX -> InX -> OutX -> InX : Selects the BDs that entered at least three times
- InX -> OutX -> InX -> OutX -> InX -> OutX : Selects the BDs that left at least three times
- InX -> OutX -> InX -> OutX -> InX -> OutX -> InX : Selects the BDs that entered at least four times
- InX -> OutX -> InX -> OutX -> InX -> OutX -> InX -> OutX : Selects the BDs that left at least four times
- OutX : Selects the BDs that have left
- $X : Selects the Bds that remained in the last slice despite removals and reinsertions
- X ~OutX : Selects the DBs that have not left.